In [25]:
''' Common import and functions  '''
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os,sys
import re
import sqlite3
from glob import glob
from pathlib import Path
try:
    from openbabel import pybel
except:
    import pybel
import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, QED
from rdkit.Chem import ChemicalFeatures
from my_toolset.my_utils import get_mol,canonic_smiles
from my_toolset.drawing_utils import show_mols
from rdkit import RDConfig
from pandarallel import pandarallel
from functools import partial
import numpy as np
import copy
pandarallel.initialize(nb_workers=64)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


#  CLM generated compound analysis

In [35]:
rootFolder="/public/home/zhangjie/Projects/MMP/pot_clm/CLModel/experiments/evaluation/test/evaluation_59"
# dfMMP=pd.read_csv(f"/public/home/zhangjie/Projects/MMP/pot_clm/Data/MedChemAct_cano_TLR7/MedChemAct_cano_TLR7_MMP_CLM.csv")
# dfMMP["Source_Mol"]=dfMMP["Source_Mol"].parallel_apply(canonic_smiles)
# dfMMP["Target_Mol"]=dfMMP["Target_Mol"].parallel_apply(canonic_smiles)

dfGen=pd.read_csv(f"{rootFolder}/generated_molecules.csv")

if 0:
    res=[]
    for idx,irow in dfGen.iterrows():
        srcCPD=canonic_smiles(irow["Source_Mol"])
        Delta_pki=re.findall(r'(\d+(?:\.\d+)?)', irow["Delta_pki"])
        Delta_pki=[float(i) for i in Delta_pki]
        Delta_pki=np.array(Delta_pki).mean()
        AC=0 ## test pair is Activity clif
        if float(Delta_pki)>2:
            AC=1
        dfTgCPD=dfMMP[dfMMP["Source_Mol"]==srcCPD]
        for igen in range(1,51):
            smi=irow[f"Predicted_smi_{igen}"]
            if not pd.isna(smi):
                smi=canonic_smiles(smi)
                if smi in dfTgCPD["Target_Mol"].to_list():
                    dfTgCPD_match=dfTgCPD[dfTgCPD["Target_Mol"]==smi]
                    Delta_pki_mean=dfTgCPD_match['Delta_pki'].mean()
                    # print(smi)
                    res.append([srcCPD,smi,Delta_pki_mean,AC])
                else:
                    print(smi)
                    res.append([srcCPD,smi,'',AC])
                    
    dfRes=pd.DataFrame(res, columns=["Source_Mol","Gen_Mol",'Delta_pki','orgAC'])
    dfRes.to_csv(f"{rootFolder}/generated_Analysis.csv", index=None)

if 1:
    dfRes=dfRes[dfRes['orgAC']==1]
    dfRes=dfRes.drop_duplicates(subset=["Source_Mol","Gen_Mol"])
    dfRes_notNull=dfRes[dfRes['Delta_pki']!='']
    print("1.5/1.6: ", len(dfRes)-len(dfRes_notNull))
    dfRes_tmp=dfRes_notNull[dfRes_notNull['Delta_pki']>2]
    print("1.1/1.2 (>2): ", len(dfRes_tmp))
    dfRes_tmp=dfRes_notNull[dfRes_notNull['Delta_pki']>1]
    print("1.1/1.2 (>1): ", len(dfRes_tmp))
    dfRes_tmp=dfRes_notNull[dfRes_notNull['Delta_pki']<1]
    print("1.1/1.2 (<1): ", len(dfRes_tmp))
    
if 0:
    # dfRes_tmp=dfRes_notNull[dfRes_notNull['Delta_pki']>2]
    dfRes_tmp=dfRes[dfRes['Delta_pki']=='']
    count=0
    for idx,irow in dfRes_tmp.iterrows():
        png,svg=show_mols([irow["Source_Mol"], irow["Gen_Mol"]])
        display(png)
        count+=1
        if count>10:
            break
        
    
        




1.5/1.6:  4004
1.1/1.2 (>2):  1029
1.1/1.2 (>1):  1824
1.1/1.2 (<1):  896
